In [27]:
import bpy
import databpy as db
import polars as pl

obj = bpy.data.objects["Cube"]
evaluated_obj = db.evaluate_object(obj)
mesh = evaluated_obj.to_mesh()

# Collect all attribute data
attribute_data = {}
for attr in mesh.attributes:
    if attr.name not in {'sharp_face', 'UVMap'} and not attr.name.startswith('.'):
        a = db.named_attribute(evaluated_obj, attr.name)
        attribute_data[attr.name] = a

# Create polars DataFrame
df = pl.DataFrame(attribute_data)

# Sort columns so "position" is first
if "position" in df.columns:
    column_order = ["position"] + [col for col in df.columns if col != "position"]
    df = df.select(column_order)

print(df)

shape: (8, 3)
┌────────────────────┬───────┬──────────┐
│ position           ┆ hello ┆ world!!! │
│ ---                ┆ ---   ┆ ---      │
│ array[f64, 3]      ┆ i64   ┆ bool     │
╞════════════════════╪═══════╪══════════╡
│ [-1.0, -1.0, -1.0] ┆ 42    ┆ true     │
│ [-1.0, -1.0, 1.0]  ┆ 42    ┆ true     │
│ [-1.0, 1.0, -1.0]  ┆ 42    ┆ true     │
│ [-1.0, 1.0, 1.0]   ┆ 42    ┆ true     │
│ [1.0, -1.0, -1.0]  ┆ 42    ┆ true     │
│ [1.0, -1.0, 1.0]   ┆ 42    ┆ true     │
│ [1.0, 1.0, -1.0]   ┆ 42    ┆ true     │
│ [1.0, 1.0, 1.0]    ┆ 42    ┆ true     │
└────────────────────┴───────┴──────────┘


In [24]:
import tempfile
import os

# Write to temporary file
temp_dir = tempfile.gettempdir()
temp_file = os.path.join(temp_dir, "mesh_attributes.csv")
df.write_csv(temp_file)
print(f"CSV written to: {temp_file}")
print(df)

ComputeError: datatype array[f64, 3] cannot be written to CSV

Consider using JSON or a binary format.

In [25]:
for attr in mesh.attributes:
    print(attr.name)

hello
position
.select_vert
world!!!
.edge_verts
.select_edge
.select_poly
sharp_face
.corner_vert
.corner_edge
UVMap


In [26]:
for attr in mesh.attributes:
    if attr.name not in {'sharp_face', 'UVMap'} and not attr.name.startswith('.'):
        a = db.named_attribute(obj, attr.name)
        print(attr.name + "\n" + str(a))

NamedAttributeError: The selected attribute 'hello' does not exist on the mesh.